In [81]:
import pandas as  pd
import matplotlib.pyplot as plt

### The next cell imports the raw data and drops
* Null columns
* Rows with disease name as null

In [65]:
patients_df = pd.read_csv("AI DATA MODIFIED.csv",encoding = "ISO-8859-1")
patients_df.drop(columns=['HEART_RATE','HEAD_CIRCUMFERENCE','UPPER_ARM_CIRCUMFERENCE'],inplace = True)
patients_df = patients_df[pd.notnull(patients_df['DISEASE_NAME'])]

C:\Users\Soham\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
patients_df.head()

,PATIENT_VISIT_ID,PATIENT_NAME,GENDER,AGE,STATE_NAME,DISTRICT_NAME,CENTER_SHORT_NAME,SYSTOLIC_BP,DIASTOLIC_BP,PULSE,RESPIRATION_RATE,BODY_TEMPERATURE,BODY_WEIGHT,HEIGHT,SPO2,SYMPTOM_ID,SYMPTOM_DESC,MEDICAL_TEST_NAME,DISEASE_NAME
0,PV000000554626,Harjas Singh,Male,4.0,Rajasthan,Bundi,Pandit Briz Sundar Sharma General Hospital Bundi,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,ECZIMA,NaN,ACD (POST APPLICATION OF DETTOL)
1,PV000000554628,Hetal Vaishnav,Male,22.5,Rajasthan,Udaipur,Satelite Hospital Hiran Magri Sec 5 Udaipur,120.0,80.0,68.0,18.0,NaN,53.0,NaN,97.0,NaN,AC URI WITH STOMATITIS,CBC~SGOT (AST),ORAL ULCER~URI
2,PV000000554632,Om Perkash,Male,26.0,Rajasthan,Bikaner,District Hospital (Satellite) Bikaner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ITCHING,NaN,"SINGLE ERYTHMATOUS PLAQUE ,ASYMPTOMATIC ON LOW..."
3,PV000000554634,Kesar Dev,Male,35.0,Rajasthan,Jhunjhunun,B D K Hospital Jhunjhunu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RINGWORM,NaN,FUC TINEA C/O ITCHING
4,PV000000554636,Paavan Swami,Male,15.2,Rajasthan,Churu,Government Hospital Ratangarh,110.0,70.0,72.0,24.0,98.6,45.0,154.0,99.0,NaN,WHITE HAIR,NaN,PREMATURE CANITIES


Vitals to be categorized:

- [X] Age
- [X] Sys and Dias BP
- [ ] Pulse
- [ ] RESPIRATION_RATE
- [ ] BODY_TEMPERATURE
- [ ] BODY_WEIGHT
- [ ] Height
- [ ] SPO2

### The next cell drops all rows with wrongly entered vitals

In [67]:
patients_df.drop(patients_df[(patients_df.SYSTOLIC_BP>230) | (patients_df.DIASTOLIC_BP>150)].index, inplace = True)
patients_df.drop(patients_df[(patients_df.SYSTOLIC_BP<80) | (patients_df.DIASTOLIC_BP<50)].index, inplace=True)
patients_df.drop(patients_df[(patients_df.BODY_TEMPERATURE<90) | (patients_df.BODY_TEMPERATURE>110)].index, inplace=True)
patients_df.drop(patients_df[patients_df.AGE>100].index, inplace=True)
patients_df.drop(patients_df[(patients_df.SPO2>100) | (patients_df.SPO2<80)].index,inplace=True)
patients_df.drop(patients_df[(patients_df.PULSE<50) | (patients_df.PULSE>120)].index,inplace=True)
patients_df.drop(patients_df[(patients_df.RESPIRATION_RATE>30) | (patients_df.RESPIRATION_RATE<13)].index,inplace=True)
patients_df.drop(patients_df[patients_df.BODY_WEIGHT>200].index,inplace=True)
patients_df.drop(patients_df[(patients_df.HEIGHT>200) | (patients_df.HEIGHT<45)].index,inplace=True)

### Binning age values:

In [68]:
bins = [x for x in range(0,101,5)]
labels = [x for x in range(1,21)]

In [69]:
patients_df['age_binned'] = pd.cut(patients_df.AGE,bins,labels=labels)

#### As we see, most of the vitals consist of null values

In [71]:
patients_df.isnull().sum()/patients_df.shape[0]*100

PATIENT_VISIT_ID      0.000000
PATIENT_NAME          0.000000
GENDER                0.000000
AGE                   0.000000
STATE_NAME            0.000000
DISTRICT_NAME         0.000000
CENTER_SHORT_NAME     0.000000
SYSTOLIC_BP          45.413184
DIASTOLIC_BP         45.397137
PULSE                33.352055
RESPIRATION_RATE     53.784471
BODY_TEMPERATURE     42.584211
BODY_WEIGHT          32.158449
HEIGHT               58.038786
SPO2                 35.903511
SYMPTOM_ID           41.566649
SYMPTOM_DESC         58.429575
MEDICAL_TEST_NAME    88.798324
DISEASE_NAME          0.000000
age_binned            0.010383
dtype: float64

### Replacing with default values if available, else median

<b>NOTE:</b> This might be risky since many patients with missing vitals may actually contain abnormal values, which the model may fail to consider. Thus, better to infer values depending on other vitals (to check later)

In [72]:
patients_df.SYSTOLIC_BP.fillna(120,inplace=True)
patients_df.DIASTOLIC_BP.fillna(80,inplace=True)
patients_df.PULSE.fillna(patients_df.PULSE.median(),inplace=True)
patients_df.RESPIRATION_RATE.fillna(patients_df.RESPIRATION_RATE.median(),inplace=True)
patients_df.BODY_TEMPERATURE.fillna(patients_df.BODY_TEMPERATURE.median(),inplace=True)
patients_df.BODY_WEIGHT.fillna(patients_df.BODY_WEIGHT.median(),inplace=True)
patients_df.HEIGHT.fillna(patients_df.HEIGHT.median(),inplace=True)
patients_df.SPO2.fillna(patients_df.SPO2.median(),inplace=True)

In [73]:
patients_df.describe()

,AGE,SYSTOLIC_BP,DIASTOLIC_BP,PULSE,RESPIRATION_RATE,BODY_TEMPERATURE,BODY_WEIGHT,HEIGHT,SPO2
count,211879.000000,211879.000000,211879.000000,211879.000000,211879.000000,211879.000000,211879.000000,211879.000000,211879.000000
mean,33.797301,119.223212,78.603595,83.112687,18.708758,97.935017,44.354730,148.598699,98.385555
std,20.402416,8.769123,5.975852,8.689792,2.127351,0.924210,14.574181,15.135122,1.569048
min,0.000000,80.000000,50.000000,50.000000,13.000000,90.000000,0.100000,45.000000,80.000000
25%,18.000000,120.000000,80.000000,78.000000,18.000000,98.000000,42.000000,151.000000,98.000000
50%,31.000000,120.000000,80.000000,82.000000,18.000000,98.000000,45.000000,151.000000,99.000000
75%,50.000000,120.000000,80.000000,85.000000,18.000000,98.200000,50.000000,151.000000,99.000000
max,100.000000,230.000000,150.000000,120.000000,30.000000,109.200000,180.000000,200.000000,100.000000


### Making categories for BP

In [74]:
patients_df['Blood pressure']=''
patients_df['Blood pressure'][(patients_df.SYSTOLIC_BP<90) | (patients_df.DIASTOLIC_BP<60)] = 'low'
patients_df['Blood pressure'][((patients_df.SYSTOLIC_BP>=90) & (patients_df.SYSTOLIC_BP<=120)) & ((patients_df.DIASTOLIC_BP>=60) & (patients_df.DIASTOLIC_BP<=80))] = 'normal'
patients_df['Blood pressure'][((patients_df.SYSTOLIC_BP>120) & (patients_df.SYSTOLIC_BP<130)) & ((patients_df.DIASTOLIC_BP>=60) & (patients_df.DIASTOLIC_BP<=80))] = 'elevated'
patients_df['Blood pressure'][((patients_df.SYSTOLIC_BP>=130) & (patients_df.SYSTOLIC_BP<140)) | ((patients_df.DIASTOLIC_BP>80) & (patients_df.DIASTOLIC_BP<90))] = 'high1'
patients_df['Blood pressure'][((patients_df.SYSTOLIC_BP>=140) & (patients_df.SYSTOLIC_BP<180)) | ((patients_df.DIASTOLIC_BP>=90) & (patients_df.DIASTOLIC_BP<120))] = 'high2'
patients_df['Blood pressure'][(patients_df.SYSTOLIC_BP>=180) | (patients_df.DIASTOLIC_BP>=120)] = 'hypertensive'

C:\Users\Soham\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Soham\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Soham\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Soham\Anaconda3\lib\

In [77]:
patients_df['Blood pressure'].value_counts(normalize=True)*100

normal          88.529302
high2            5.073179
high1            4.752241
elevated         1.148769
low              0.266662
hypertensive     0.229848
Name: Blood pressure, dtype: float64

#### Working on data from Orissa since it has the max no. of consultations 

In [42]:
patients_df.STATE_NAME.value_counts(normalize=True)*100

Orissa           56.037644
Rajasthan        41.498686
West Bengal       1.292719
Gujarat           0.593735
Chhattisgarh      0.229376
Jharkhand         0.155277
Bihar             0.154805
Uttar Pradesh     0.029262
Delhi             0.008495
Name: STATE_NAME, dtype: float64

In [33]:
orissa_df = patients_df[patients_df.STATE_NAME=='Orissa']

In [34]:
orissa_df.shape

(118732, 20)

In [35]:
orissa_df.describe()

,AGE,SYSTOLIC_BP,DIASTOLIC_BP,PULSE,RESPIRATION_RATE,BODY_TEMPERATURE,BODY_WEIGHT,HEIGHT,SPO2
count,118732.000000,118732.000000,118732.000000,118732.000000,118732.000000,118732.000000,118732.000000,118732.000000,118732.000000
mean,33.962603,123.676380,80.615967,84.552412,18.912635,97.862317,42.566670,146.284889,98.070444
std,21.512701,13.387587,8.618599,10.311345,2.363849,1.145696,16.984812,18.758125,1.904339
min,0.000000,80.000000,50.000000,50.000000,13.000000,90.000000,0.100000,45.000000,80.000000
25%,15.000000,120.000000,78.000000,78.000000,18.000000,97.700000,35.000000,146.000000,98.000000
50%,32.950000,120.000000,80.000000,82.000000,18.000000,98.000000,45.000000,151.000000,99.000000
75%,52.000000,140.000000,90.000000,89.000000,19.000000,98.400000,54.000000,154.000000,99.000000
max,100.000000,230.000000,150.000000,120.000000,30.000000,109.200000,170.000000,200.000000,100.000000


In [36]:
orissa_df.isnull().sum()/orissa_df.shape[0]*100

PATIENT_VISIT_ID      0.000000
PATIENT_NAME          0.000000
GENDER                0.000000
AGE                   0.000000
STATE_NAME            0.000000
DISTRICT_NAME         0.000000
CENTER_SHORT_NAME     0.000000
SYSTOLIC_BP           0.000000
DIASTOLIC_BP          0.000000
PULSE                 0.000000
RESPIRATION_RATE      0.000000
BODY_TEMPERATURE      0.000000
BODY_WEIGHT           0.000000
HEIGHT                0.000000
SPO2                  0.000000
SYMPTOM_ID           42.035003
SYMPTOM_DESC         57.960786
MEDICAL_TEST_NAME    88.065559
DISEASE_NAME          0.000000
age_binned            0.010107
dtype: float64

In [39]:
orissa_df[orissa_df.SYMPTOM_DESC.notnull()].shape

(49914, 20)

In [40]:
orissa_df[orissa_df.SYMPTOM_DESC.notnull()].to_csv('orissa_symptom_desc_notnull.csv')

In [21]:
symptoms = pd.read_csv('SYMPTOM MASTER.csv',encoding = "ISO-8859-1")

In [108]:
symptoms

,SYMPTOM_ID,SYMPTOM_NAME,GENDER
0,SYMP04148,Increased Urinary Infection,C
1,SYMP04158,Increased Vocal Resonance,C
2,SYMP00998,indentation on either side of the nasal tip,C
3,SYMP00130,Excessive Burping/ Belching,C
4,SYMP00135,Indigestion,C
5,SYMP03062,Eructation,C
6,SYMP03093,Gas,C
7,SYMP00962,Indurated Lower Lip,C
8,SYMP00803,Induration,C
9,SYMP03045,Dependent Edema,C


In [50]:
symptoms[symptoms.SYMPTOM_ID == 'SYMP00756']

,SYMPTOM_ID,SYMPTOM_NAME,GENDER
1557,SYMP00756,Fever,C
